# Is it a bird?

This is an book to show how fastai can be trained and used to classify images.

## Train the model

In [ ]:
import socket,warnings
try:
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
except socket.error as ex: raise Exception("STOP: No internet. Click '>|' in top right and set 'Internet' switch to on")

%pip install -Uqq fastai "duckduckgo_search>=6.2"

In [3]:
from duckduckgo_search import DDGS
from fastcore.all import *

def search_images(keywords, max_images=200): return L(DDGS().images(keywords, max_results=max_images)).itemgot('image')
import time, json

In [ ]:
#NB: `search_images` depends on duckduckgo.com, which doesn't always return correct responses.
#    If you get a JSON error, just try running it again (it may take a couple of tries).
urls = search_images('bird photos', max_images=1)
urls[0]

In [ ]:
from fastdownload import download_url
dest = 'bird.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
download_url(search_images('forest photos', max_images=1)[0], 'forest.jpg', show_progress=False)
Image.open('forest.jpg').to_thumb(256,256)

In [8]:
searches = 'forest','bird'
path = Path('bird_or_not')

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    time.sleep(5)
    resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

In [ ]:
type,_,probs = learn.predict(PILImage.create('bird.jpg'))
print(f"This is a: {type}.")
print(f"Probability it's a {type}: {probs[0]:.4f}")

In [ ]:
type,_,probs = learn.predict(PILImage.create('forest.jpg'))
print(f"This is a: {type}.")
print(f"Probability it's a forest: {probs[1]:.4f}")

## Prepare the model for use in an application

In [15]:
# export the model for use in an application (export.pkl)
learn.export()

In [16]:
# load the model for use in an application
learn_inf = load_learner('export.pkl')

In [ ]:
# print the model's vocabulary
learn_inf.dls.vocab

In [ ]:
learn_inf.predict('bird.jpg')

## Create an application to use the model

In [ ]:
! pip install -Uqq fastai
from fastai import *
from fastai.vision.widgets import *

btn_upload = widgets.FileUpload()
btn_upload


In [ ]:
import io

# create a button to do classification
btn_run = widgets.Button(description='Classify')
btn_run

# display the prediction via labels
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

def on_click_classify(change):
    uploaded_file = btn_upload.value[0]
    img = PILImage.create(io.BytesIO(uploaded_file.content.tobytes()))
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)


In [ ]:
VBox([widgets.Label('Select your image!'), 
      btn_upload, btn_run, out_pl, lbl_pred])

In [ ]:
!pip install voila
!jupyter server extension enable --sys-prefix voila